In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import json
import unicodedata

In [2]:
doc = pd.read_csv('data/ica_data.csv', encoding='ISO-8859-1', error_bad_lines=False, delimiter=';')
doc.head()

,ID,Timestamp,Tweet,Keluhan,Respon,Bukan Keluhan/Respon,Topik Umum,Topik Spesifik,Lokasi,Waktu,Unnamed: 10
0,"6,78845E+17","21 Des 2015, 02.50.25 PM",@EL_Atheos @ridwankamil ya mungkin karena pere...,NaN,NaN,Ya,Bukan Keluhan,-,-,-,NaN
1,"6,78846E+17","21 Des 2015, 02.53.02 PM",@ridwankamil @dbmpkotabdg kang teman saya tert...,Ya,NaN,NaN,Lingkungan Hidup,pohon tumbang,Jalan Sangkuriang depan Polsek Coblong,-,NaN
2,"6,78847E+17","21 Des 2015, 02.57.55 PM",Di tribun jabar biasanya suka di post agenda k...,NaN,NaN,Ya,Bukan Keluhan,-,-,-,NaN
3,"6,78847E+17","21 Des 2015, 02.58.54 PM",@dbmpkotabdg RT @fajriattack: Lapor pak @ridwa...,Ya,NaN,NaN,Infrastruktur,lampu penerangan jalan umum,Jalan depan Kampus LPKIA,-,NaN
4,"6,78849E+17","21 Des 2015, 03.06.49 PM","@diskamtam bapak/ibu mau tanya, kalo pemelihar...",NaN,NaN,Ya,Bukan Keluhan,-,-,-,NaN


In [3]:
data = doc.loc[:, 'Tweet':'Bukan Keluhan/Respon']
data.head()

,Tweet,Keluhan,Respon,Bukan Keluhan/Respon
0,@EL_Atheos @ridwankamil ya mungkin karena pere...,NaN,NaN,Ya
1,@ridwankamil @dbmpkotabdg kang teman saya tert...,Ya,NaN,NaN
2,Di tribun jabar biasanya suka di post agenda k...,NaN,NaN,Ya
3,@dbmpkotabdg RT @fajriattack: Lapor pak @ridwa...,Ya,NaN,NaN
4,"@diskamtam bapak/ibu mau tanya, kalo pemelihar...",NaN,NaN,Ya


In [4]:
# data = data.fillna('0')
# data['Keluhan'] = data['Keluhan'].map({'Ya':1})
# data['Respon'] = data['Respon'].map({'Ya':1})
# data['Bukan Keluhan/Respon'] = data['Bukan Keluhan/Respon'].map({'Ya':1})

In [5]:
# data_text = data.loc[:, 'Tweet']
data_tweet = data.loc[:, 'Tweet']

In [6]:
# data_keluhan = data.loc[:, 'Keluhan']
# data_keluhan[0:3]

In [7]:
# get index with "Ya" value in 3 column
temp = data['Keluhan'].values.tolist()
idx_keluhan = [i for i, x in enumerate(temp) if x == 'Ya']

temp = data['Respon'].values.tolist()
idx_respon = [i for i, x in enumerate(temp) if x == 'Ya']

temp = data['Bukan Keluhan/Respon'].values.tolist()
idx_notboth = [i for i, x in enumerate(temp) if x == 'Ya']

# create 1 column for label
data_label = []
for i in range (0,len(data)):
    if i in idx_keluhan:
        data_label.append(0)
    elif i in idx_respon:
        data_label.append(1)
    elif i in idx_notboth:
        data_label.append(2)
data_label

[2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 1,
 0,
 2,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 2,
 2,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 1,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 2,
 2,
 2,
 0,
 1,
 1,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 1,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,


In [8]:
import requests
import re
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [9]:
def removeURL(sent):
    # remove username
    sent = re.sub('RT @[^\s]+','',sent)
    sent = re.sub('@[^\s]+','',sent)
    sent = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', sent, flags=re.MULTILINE)
    sent = re.sub('http[s]?', '', sent, flags=re.MULTILINE)
    return sent

In [25]:
def normalization(sent):
    # text normalization
    header = {
        'Content-Type' : 'application/json',
        'x-api-key' : 'PGSpcyOiDPqdlNmQ1kd66p59qhTDuLUOCqdJk4sF'
    }
    req = requests.post("https://api.prosa.ai/v1/normals", headers=header, json = { 'text' : sent})
    res = req.json()
    return res['text']
#     print(res)

In [11]:
def stemmer(sent):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(sent)

In [12]:
def tokenizer(sent):
    words = word_tokenize(sent)
#     print (words)
    return words

In [34]:
temp = '|ya mungkin karena perempuan bj merah yang paling kelihatan karena berada di depan, laki-laki juga sering jadi bahan joke RK|kang teman saya tertimpa pohn di jalan sangkuriang depan polsek coblong tolong ditertibkan pohon yang sudah lapuknuhun'
temp.split('|')

['',
 'ya mungkin karena perempuan bj merah yang paling kelihatan karena berada di depan, laki-laki juga sering jadi bahan joke RK',
 'kang teman saya tertimpa pohn di jalan sangkuriang depan polsek coblong tolong ditertibkan pohon yang sudah lapuknuhun']

In [75]:
n_tweet_per_call = 10 # jumlah tweet untuk setiap call normalisasi
n_call = (len(data)/1000) + 1

          
for i in range (0, n_call):
    k = i*n_tweet_per_call
    str_temp = ''
          
    for j in range (i*n_tweet_per_call, (i+1)*n_tweet_per_call):
        str_temp = str_temp + " | " + data_tweet[j]
    
    str_temp = removeURL(str_temp)
    text = normalization(str_temp)
    text = text.encode('ascii', 'ignore')
    text = stemmer(text)
    temp = text.split('|')
    print temp
          

['ya mungkin karena perempuan bj merah yang paling lihat karena ada di depan laki juga sering jadi bahan joke rk kang teman saya timpa pohn dijln sangkuriang depan polsek coblong tolong tertib pohon yang sudah lapuknuhun di tribun jabar biasa suka di pos agenda kang emil tapi tidak tiap hari sih kalau saya bandung bisa temu kang di mana lapor pak lampu terang jalan depan kampus lpkia ngaplek bapak ibu mau tanya kalau pelihara taman yang banyak dibagun sekarang bagaimana nanti pa tempat kerja saya tanggal 25 tetap masuk bagaimana itu atur pa kalau misal enggak masuk potong gaji 00106700466 punten sudah minggu air pdam tidak alir mohon bantu segera anak sma yang tahu curi hem di bip hari ini pak nyata sudah ada warga yang konfirm ke pdam kata air tidak akan alir lama 1 bulan karena ada baik kang jalan di muararajeun baru baru bulan dibenerin sudah rusak lagi yang kerja seperti enggak serius']
['didinyamah kieu bosss ku maha pak daerah sapharuhai yeuh kang parkir liar seeur pis atuhda- - 

In [71]:
temp = text.split('|')
temp

(len(data)/1000) + 1

8

In [21]:
# data_temp = []
# for tweet in data_tweet:
#     tweet = removeURL(tweet)
#     data_temp.append(tweet)
# data_tweet = data_temp

In [16]:
# data_temp = []
# for tweet in data_tweet:
#     tweet = stemmer(tweet)
#     data_temp.append(tweet)
# data_tweet = data_temp

In [17]:
# data_temp = []
# for tweet in data_tweet:
#     tweet = tokenizer(tweet)
#     data_temp.append(tweet)
# data_tweet = data_temp

In [18]:
model = Word2Vec(data_tweet, min_count=1)
model.train(data_tweet, total_examples=1, epochs=1)
x = model.wv

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
text_train, text_test, y_train, y_test = train_test_split(data_text, y, test_size=0.25, random_state=42)

NameError: name 'data_text' is not defined

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)